# Denoising Pipeline Development

In [ ]:
%load_ext autoreload
%autoreload 2

import fibsem
import salami
from fibsem import utils, acquire, alignment, calibration, milling
from fibsem.structures import BeamType, FibsemPatternSettings, FibsemMillingSettings, FibsemPattern
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import logging

from salami.structures import SalamiSettings, run_salami, create_sweep_parameters



In [ ]:
microscope, settings = utils.setup_session()


In [ ]:
eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(eb_image.data, cmap="gray")
ax[1].imshow(ib_image.data, cmap="gray")
plt.show()


### Move Raw Images to a New Directory

In [ ]:
%load_ext autoreload
%autoreload 2

import glob
import os
from pathlib import Path


path = "/home/patrick/github/salami/salami/denoise/raw_imgs"
slow_filenames = sorted(glob.glob(os.path.join(path, "*5e-06*.tif")))
fast_filenames = sorted(glob.glob(os.path.join(path, "*5e-07*.tif")))


from pprint import pprint
raw_path = "/home/patrick/github/salami/data/20230310/raw"


os.makedirs(os.path.join(raw_path, "slow"), exist_ok=True)
os.makedirs(os.path.join(raw_path, "fast"), exist_ok=True)

for fname in fast_filenames:

    idx = int(os.path.splitext(os.path.basename(fname))[0].split("_")[1])
    print(os.path.join(raw_path, "fast",  f"{idx:06d}.tif"))

    # copy file with os
    # os.system(f"cp {fname} {os.path.join(raw_path, 'fast',  f'{idx:06d}.tif')}")




In [ ]:

raw_path = "/home/patrick/github/salami/data/20230310/raw"

fast_filenames = sorted(glob.glob(os.path.join(raw_path, "fast", "*.tif")))
slow_filenames = sorted(glob.glob(os.path.join(raw_path, "slow", "*.tif")))


from pprint import pprint

# pprint(fast_filenames)

import math

from salami.denoise.inference import get_window, get_index, get_closest_index

window_size = 5
for idx, fname in enumerate(fast_filenames):

    # idx = get_index(fname) # this is based on the fname which means nothing
    window = get_window(fast_filenames, idx, window_size=window_size)
    if len(window) < window_size:
        continue
    # closest = get_closest_index(window, idx)
    # print(f"index: {idx}, closest: {get_index(closest)}")
    print(f"index: {idx}, window: {[get_index(f) for f in window]}")
    # print(window)
    

## Segmentation Diagnostics

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from salami.segmentation import segmentation as sseg

path = "/home/patrick/github/salami/salami/output/seg"

stats = sseg.calc_seg_diagnostic(path, plot=True)

In [ ]:
print(stats["counts"].shape)

In [ ]:
df = sseg.calculate_diag_df(stats)
display(df)

In [ ]:
# plot the counts

# get column names except fname

labels = df.columns[:-1]
df.plot(x="fname", y=labels, kind="line", figsize=(10,5)) 
# df.plot(x="fname", y=stats["labels"], kind="line", figsize=(10,5))

## WatchDog Testing

In [ ]:
%load_ext autoreload
%autoreload 2

import glob
import os
from pathlib import Path


path = "/home/patrick/github/salami/salami/denoise/raw_imgs"
# slow_filenames = sorted(glob.glob(os.path.join(path, "*5e-06*.tif")))
fast_filenames = sorted(glob.glob(os.path.join(path, "*5e-07*.tif")))


from pprint import pprint
raw_path = "/home/patrick/github/salami/salami/output2/raw"

# os.makedirs(os.path.join(raw_path, "slow"), exist_ok=True)
os.makedirs(raw_path, exist_ok=True)
import time
for fname in fast_filenames:

    idx = int(os.path.splitext(os.path.basename(fname))[0].split("_")[1])
    print(os.path.join(raw_path, f"{idx:06d}.tif"))

    # copy file with os
    os.system(f"cp {fname} {os.path.join(raw_path, f'{idx:06d}.tif')}")

    time.sleep(0.5)


In [ ]:
# simulate latency of 45sec vs 12 sec
# plot a graph of 1 image per 12 sec vs 1 image per 45 sec

# 1 per 12
# 1 per 45

fps12 = 1 / 12 # frames per seconds
fps45 = 1 / 45 # frames per seconds 

print(fps12, fps45)

import numpy as np
import matplotlib.pyplot as plt

x = np.arange(0, 1000, 1)
y1 = fps12 * x
y2 = fps45 * x

plt.plot(x, y1, label="12 spf")
plt.plot(x, y2, label="45 spf")
# difference
plt.plot(x, y1 - y2, label="difference")
plt.xlabel("number of seconds")
plt.ylabel("number of images")
plt.legend()
plt.show()

In [ ]:
from salami.denoise.frame import *
import matplotlib.pyplot as plt

# how to calculate FRC between to images & calculate PSNR

# load image, split in half save files
from fibsem.structures import FibsemImage
import tifffile as tff

import glob
path = "/home/patrick/github/data/salami/analysis/2023-04-14-07-05-07PM/data"
filenames = sorted(glob.glob(os.path.join(path, "*.tif")))

for fname in filenames:


    img = FibsemImage.load(fname)

    # split in half
    img1 = img.data[:, :img.data.shape[1]//2]
    img2 = img.data[:, img.data.shape[1]//2:]

    # save
    tmp_dir = os.path.join(os.path.dirname(fname), "tmp")
    os.makedirs(tmp_dir, exist_ok=True)

    fname1 = os.path.join(tmp_dir, "tmp1.tif")
    fname2 = os.path.join(tmp_dir, "tmp2.tif")
    tff.imwrite(fname1, img1)
    tff.imwrite(fname2, img2)


    # read
    img1 = Frame(filename=fname1)
    img2 = Frame(filename=fname2)

    # remove borders
    border = 128
    img1 = img1.clip(border, border, border, border)
    img2 = img2.clip(border, border, border, border)

    # PSNR
    psnr = img1.calcPSNR(img2)

    # FRC
    # image normalization, masking, Fourier transformation
    img1.normalize()
    img1.taperEdges(0.95)  # rectangular Tukey windowing (= cosine edge)
    img1.fft()

    img2.normalize()
    img2.taperEdges(0.95)  # rectangular Tukey windowing (= cosine edge)
    img2.fft()

    # actual calculation & plot
    g, frc = img1.calcFRC(img2)


    # subplot image and metric
    import matplotlib.pyplot as plt
    print(f"{os.path.basename(fname)}, {img.data.shape}, PSNR: {psnr:.2f}")

    fig, ax = plt.subplots(1,2, figsize=(10,5))
    ax[0].imshow(img.data, cmap="gray")
    ax[1].plot(g, frc)
    plt.show()

# TODO: port this to not use the Frame class
# TODO: getting negative values?
# TODO: need to understand what is happening


## Analysis Pipeline

In [ ]:
%load_ext autoreload
%autoreload 2

from salami import analysis as sa
from fibsem.structures import FibsemImage

import os
import glob

from scipy import ndimage
import matplotlib.pyplot as plt

import pandas as pd
import os
import glob
from tqdm import tqdm
import numpy as np



In [ ]:

# x axis = 1 / nm


path = "/home/patrick/github/data/salami/analysis/2023-04-14-07-05-07PM/data"
df = sa.run_salami_analysis_frc(path, plot=True, show=True)



In [ ]:
df = pd.read_csv(os.path.join(path, "metrics.csv"))

# parameters + metrics
df = sa.join_df(path)
display(df)


In [ ]:
import plotly.express as px


# convert dwell time to category
df["dwell_time"] = df["dwell_time"].astype("category")

# plot 3d scatter plot with current, pixelsize and int_05 color by dwell time
fig = px.scatter_3d(df, x="current", 
                    y="pixelsize", 
                    z="int_05", 
                    color="dwell_time", 
                    opacity=0.5)
fig.show()


# plot 3d scatter plot with current, pixelsize and int_0143 color by dwell time
fig = px.scatter_3d(df, x="current", 
                    y="pixelsize", 
                    z="int_0143", 
                    color="dwell_time", 
                    opacity=0.5)
fig.show()

In [ ]:

# plot line plot with pixelsize and int_05 color by dwell time, marker by current

fig = px.line(df, x="pixelsize", y="int_05", 
              color="dwell_time", 
              line_group="current",
              line_dash="current", 
              hover_name="basename")

# set title
fig.update_layout(title="FRC 0.5")

fig.show()

fig = px.line(df, x="pixelsize", y="int_0143", 
              color="dwell_time", 
              line_group="current",
              line_dash="current", 
              hover_name="basename")
# set title
fig.update_layout(title="FRC 0.143")
fig.show()


In [ ]:


# group by current
df_group = df.groupby("current").mean().reset_index()


## Experiment Management


In [ ]:
%load_ext autoreload
%autoreload 2

from salami.structures import Experiment, SalamiSettings, SalamiImageSettings

from fibsem.structures import ImageSettings, BeamType, BeamSettings, FibsemDetectorSettings, MicroscopeState
from fibsem import utils as futils
from fibsem.patterning import FibsemMillingStage

import os



In [ ]:
microscope, settings = futils.setup_session(manufacturer="Demo")


In [ ]:


PATH = os.getcwd()
exp = Experiment(path=PATH, name="salami")

beam_type=  BeamType.ELECTRON
exp.settings = SalamiSettings(
    n_steps=10,
    step_size=150e-9,
    image = [SalamiImageSettings(
        settings.image,
        BeamSettings(beam_type=beam_type), 
        FibsemDetectorSettings()
        )
    ],
    mill = FibsemMillingStage()
)

print(exp)

exp.settings.mill.pattern.protocol ={"width":50e-6, "height":5e-6, "depth": 10e-6, "rotation": 0.0}

In [ ]:
print(exp.settings.mill.pattern.point)


In [ ]:
from pprint import pprint
pprint(exp.settings)

In [ ]:
exp.save()
print(exp)

In [ ]:
LOAD_PATH = "../salami/salami/salami.yaml"
exp2 = Experiment.load(LOAD_PATH)
print(exp2)

In [ ]:
from salami.core import run_salami

run_salami(microscope, settings, exp.settings)

In [ ]:
settings.image

In [ ]:
from salami.core import load_protocol
from fibsem.utils import load_yaml

protocol = load_yaml("../salami/protocol/protocol.yaml")

ss = load_protocol(protocol)

pprint(ss.image[0].image)
pprint(ss.mill.pattern)
print(ss.mill.milling)

In [ ]:
pprint(protocol)

In [ ]:
pprint(ss.mill.pattern.protocol)

ss.mill.pattern.define(ss.mill.pattern.protocol)

In [ ]:
run_salami(microscope, settings, ss)